<a href="https://colab.research.google.com/github/mhpbreugem/BBP/blob/main/bbp_2s.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [151]:
using Distributions, Random, NLsolve, LinearAlgebra, Printf, Optim, BenchmarkTools, KrylovKit

2 stock code with symmetrixal distributions and zero correlation between stock payoff

In [152]:
# Chapter 0: Parameters

# Economic parameters
const β = 0.95
const γ = 4.0
const e0 = 5.0
const EΠ = 1.0
const Π0 = 1.0
const θXbar = 1.0
const θYbar = 1.0
const μθX = 0.5
const σθX = 0.2
const μθY = 0.5
const σθY = 0.1
const σΠ = 0.4
const W0 = e0

const ΞЮ = [repeat([x], 1) for x in range(0.1, stop=1.0, length=10)]

# Grid parameters
const Nσπ = 3
const NNσπ = 1
const Nπ = 2 * Nσπ * NNσπ + 1  # number of grid points for payoff

const NσΠ = 2
const NNσΠ = 1
const NΠ = 2 * NσΠ * NNσΠ + 1  # number of grid points for payoff

const NθX = 3
const NθY = 3
const NσθX = 1
const NσθY = 1

# Derived midpoints
const μπi = (Nπ + 1) ÷ 2
const μΠi = (NΠ + 1) ÷ 2
const μθXi = (NθX + 1) ÷ 2

# Total number of states
const NS = Nπ
const NN = NθX *NθX * NθY * NΠ* NΠ

675

In [153]:
# Chapter 1: Grid Construction

# Conjectured Payoffs
const ππ = collect(range(EΠ - Nσπ*σΠ, EΠ + Nσπ*σΠ, length=Nπ))
const πω = pdf.(Normal(EΠ, σΠ), ππ)
const πΩ = πω ./ sum(πω)

# Payoff Parameters
const Δ1 = Π0
const Δ2 = ππ

# True Payoffs
const ΞΠ = EΠ .+ σΠ .* range(-NσΠ, NσΠ, length=NΠ)
const ππ_to_index = Dict(round(ππ[πi], digits=8) => πi for πi in 1:Nπ)
const ΞΠi = [ππ_to_index[round(ΞΠ[Πi], digits=8)] for Πi in 1:NΠ]

# Signals
const S = ππ
const ϵ = S .- ππ'

@inline function ψϵω(Ю, ϵx)
    σϵ = 1 / sqrt(Ю)
    pdf(Normal(0.0, σϵ), ϵx)
end

function φSΩπ(Ю)
    σϵ = 1 / sqrt(Ю[1])
    SΩ = [pdf(Normal(π, σϵ), s) for π in ππ, s in S]
    SΩnorm = SΩ ./ sum(SΩ, dims=2)
    return SΩnorm
end

# Noise Trader Demand
const ΞθX = μθX .+ σθX .* range(-NσθX, NσθX, length=NθX)
const ΞθY = μθY .+ σθY .* range(-NσθY, NσθY, length=NθY)

const Σθ = [σθX^2 0 0; 0 σθX^2 0; 0 0 σθY^2]
const dist = MvNormal([μθX, μθX, μθY], Σθ)
θXYωω(xA, xB, y) = pdf(dist, [xA, xB, y])

θXYωω (generic function with 1 method)

In [158]:
# Chapter 2: Learning

function φΞΩI(Ю, XAΣ, XBΣ, YΣ)
    ψϵ = ψϵω.(Ю, ϵ)                             # NS × Nπ
    θXY = θXYωω.(θXbar .- XAΣ, θXbar .- XBΣ, θYbar .- YΣ)  # Nπ × Nπ

    ψπ = ψϵ .* transpose(πΩ)                     # NS × Nπ
    ψπ_T = transpose(ψπ)                        # Nπ × NS

    A = reshape(ψπ_T, Nπ, 1, NS, 1)             # Nπ × 1 × NS × 1
    B = reshape(ψπ_T, 1, Nπ, 1, NS)             # 1 × Nπ × 1 × NS
    Θ = reshape(θXY, Nπ, Nπ, 1, 1)              # Nπ × Nπ × 1 × 1

    ΞΩI = A .* B .* Θ                           # Nπ × Nπ × NS × NS
    ΞΩI ./= sum(ΞΩI, dims=(1, 2))               # Normalize each (s1, s2) slice

    return ΞΩI
end

φΞΩI (generic function with 1 method)

In [271]:
# Chapter 3: System of Equations

function φSYS(Ж, Ю, Б)
    # 1. Unpack inputs
    XA = reshape(Ж[1:NS^2], NS, NS)
    XB = reshape(Ж[NS^2+1:2NS^2], NS, NS)
    C1 = reshape(Ж[2NS^2+1:3NS^2], NS, NS)
    PXA, PXB, PY = Ж[3NS^2+1:3NS^2+3]
    ΠAi, ΠBi = Б[1], Б[2]
    θXA, θXB, θY = ΞθX[Б[3]], ΞθX[Б[4]], ΞθY[Б[5]]

    # 2. Aggregates
    SΩπ = φSΩπ(Ю)
    XAΣ = SΩπ * XA * transpose(SΩπ)
    XBΣ = SΩπ * XB * transpose(SΩπ)
    C1Σ = SΩπ * C1 * transpose(SΩπ)
    YΣ = (ΣW0 .- PXA .* XAΣ .- PXB .* XBΣ .- C1Σ) ./ PY

    # 3. Learning tensor Ω
    Ω = φΞΩI(Ю, XAΣ, XBΣ, YΣ)

    # 4. Final consumption c2(π₁, π₂, s₁, s₂)
    Cbase = (W0 .- PXA .* XA .- PXB .* XB .- C1) ./ PY
    c2 = reshape(Cbase, 1, 1, NS, NS) .+
         reshape(XA, 1, 1, NS, NS) .* reshape(Δ2, Nπ, 1, 1, 1) .+
         reshape(XB, 1, 1, NS, NS) .* reshape(Δ2, 1, Nπ, 1, 1)

    # 5. Utility weights and parameter vectors
    Z = exp.(-γ .* c2)
    KA = (-PXA / PY) .+ Δ2
    KB = (-PXB / PY) .+ Δ2
    KA_flat = repeat(KA, Nπ)
    KB_flat = repeat(KB, inner=Nπ)

    # 6. Flattened matrix contractions
    ZO = Z .* Ω   # shape: Nπ × Nπ × NS × NS
    KA_tensor = reshape(KA, Nπ, 1, 1, 1)
    KB_tensor = reshape(KB, 1, Nπ, 1, 1)

    FOCXA = β .* vec(sum(ZO .* KA_tensor, dims=(1,2)))  # NS² vector
    FOCXB = β .* vec(sum(ZO .* KB_tensor, dims=(1,2)))  # NS² vector
    FOCC1 = exp.(-γ .* C1)[:] .+ β .* vec(sum(ZO, dims=(1,2))) .* (-1 / PY) # NS² vector

    # 7. Market clearing
    MCXA = XAΣ[ΞΠi[ΠAi], ΞΠi[ΠBi]] - (θXbar - θXA)
    MCXB = XBΣ[ΞΠi[ΠAi], ΞΠi[ΠBi]] - (θXbar - θXB)
    MCY  = YΣ[ΞΠi[ΠAi], ΞΠi[ΠBi]] - (θYbar - θY)

    # 8. Return full residual system
    #return XAΣ
    return vcat(FOCXA, FOCXB, FOCC1, MCXA, MCXB, MCY)
end


φSYS (generic function with 1 method)

In [235]:
# Chapter 4: Zero-Info Reference
const W0ξ = W0

function φSYSξ(Жξ, Б)
    # 1. Extract prices
    PXAξ, PXBξ, PYξ = Жξ[1:3]

    # 2. Retrieve θ-shocks
    θXA = ΞθX[Б[3]]
    θXB = ΞθX[Б[4]]
    θY  = ΞθY[Б[5]]

    # 3. Direct market clearing values (XAξ, XBξ are scalars)
    XAξ = θXbar - θXA
    XBξ = θXbar - θXB
    Yξ = θYbar - θY

    # 4. Residual consumption (scalar)
    C1ξ = W0ξ - PXAξ * XAξ - PXBξ * XBξ - PYξ * Yξ

    # 5. Construct final consumption grid (Nπ × Nπ)
    C2ξ = Yξ .+ XAξ .* reshape(Δ2, Nπ, 1) .+ XBξ .* reshape(Δ2, 1, Nπ)

    # 6. Utility values on the grid
    U = exp.(-γ .* C2ξ)  # Nπ × Nπ

    # 7. Joint probability weights
    ππ = πΩ * transpose(πΩ)  # outer product — Nπ × Nπ

    # 8. FOCs (expectation over ππ)
    FOCXA = β * sum(ππ .* U .* ((-PXAξ / PYξ) .+ reshape(Δ2, Nπ, 1)))
    FOCXB = β * sum(ππ .* U .* ((-PXBξ / PYξ) .+ reshape(Δ2, 1, Nπ)))
    FOCC1 = exp(-γ * C1ξ) + β * sum(ππ .* U) * (-1.0 / PYξ)

    return vcat(FOCXA, FOCXB, FOCC1)
end

φSYSξ (generic function with 1 method)

In [239]:
# Chapter 5: Starting Point

function φЖ0(Б)
    # 1. Extract shocks
    ΠAi, ΠBi = Б[1], Б[2]
    θXA = ΞθX[Б[3]]
    θXB = ΞθX[Б[4]]
    θY  = ΞθY[Б[5]]

    # 2. Solve zero-info fixed point
    solξ = nlsolve(Жξ -> φSYSξ(Жξ, Б), [0.6, 0.6, 0.7])
    PXAξ, PXBξ, PYξ = solξ.zero

    # 3. Recover XAξ, XBξ, Yξ
    XAξ = θXbar - θXA
    XBξ = θXbar - θXB
    Yξ  = θYbar - θY

    # 4. Compute constant C1ξ from residual budget
    C1ξ = W0ξ - PXAξ * XAξ - PXBξ * XBξ - PYξ * Yξ

    # 5. Fill full Ж vector
    Ж = zeros(3NS^2 + 3)
    Ж[1:NS^2]               .= XAξ
    Ж[NS^2+1:2NS^2]         .= XBξ
    Ж[2NS^2+1:3NS^2]        .= C1ξ
    Ж[3NS^2+1:3NS^2+3]      .= [PXAξ, PXBξ, PYξ]

    Ж = Ж.+ 1e-3 .* randn(length(Ж))
    return Ж
end

φЖ0 (generic function with 1 method)

In [ ]:
# Chapter 6: Solution over extended 5D grid

sol_zeros = Array{Vector{Float64}, 5}(undef, NΠ, NΠ, NθX, NθX, NθY)

for ΠAi in 1:NΠ
    for ΠBi in 1:NΠ
        for θXAi in 1:NθX
            for θXBi in 1:NθX
                for θYi in 1:NθY

                    # 1. Pack parameter bundle for this grid point
                    Б = [ΠAi, ΠBi, θXAi, θXBi, θYi]

                    # 2. Generate starting point using zero-info fixed point
                    Ж_ini = φЖ0(Б)

                    # 3. Iterate over Ю values for continuation (annealing)
                    for Ю_value in range(0.1, stop=1, length=6)
                        print("\rΠAi=$ΠAi ΠBi=$ΠBi θXAi=$θXAi θXBi=$θXBi θYi=$θYi Ю=$Ю_value")
                        flush(stdout)

                        sol = nlsolve(Ж -> φSYS(Ж, [Ю_value], Б), Ж_ini)

                        if sol.f_converged
                            Ж_ini = sol.zero
                        else
                            @warn "No convergence at ΠAi=$ΠAi ΠBi=$ΠBi θXAi=$θXAi θXBi=$θXBi θYi=$θYi Ю=$Ю_value"
                            break
                        end
                    end

                    # 4. Store result
                    sol_zeros[ΠAi, ΠBi, θXAi, θXBi, θYi] = Ж_ini
                end
            end
        end
    end
end


ΠAi=1 ΠBi=3 θXAi=3 θXBi=2 θYi=2 Ю=0.46

In [262]:
TT0 = [
    0.671248,0.671248,0.671248,0.671248,0.671248,0.671248,0.671248,
    0.681248,0.681248,0.681248,0.681248,0.681248,0.681248,0.681248,
    0.691248,0.691248,0.691248,0.691248,0.691248,0.691248,0.691248,
    0.701248,0.701248,0.701248,0.701248,0.701248,0.701248,0.701248,
    0.711248,0.711248,0.711248,0.711248,0.711248,0.711248,0.711248,
    0.721248,0.721248,0.721248,0.721248,0.721248,0.721248,0.721248,
    0.731248,0.731248,0.731248,0.731248,0.731248,0.731248,0.731248,
    0.671248,0.681248,0.691248,0.701248,0.711248,0.721248,0.731248,
    0.671248,0.681248,0.691248,0.701248,0.711248,0.721248,0.731248,
    0.671248,0.681248,0.691248,0.701248,0.711248,0.721248,0.731248,
    0.671248,0.681248,0.691248,0.701248,0.711248,0.721248,0.731248,
    0.671248,0.681248,0.691248,0.701248,0.711248,0.721248,0.731248,
    0.671248,0.681248,0.691248,0.701248,0.711248,0.721248,0.731248,
    0.671248,0.681248,0.691248,0.701248,0.711248,0.721248,0.731248,
    1.88941,1.8923,1.89524,1.89821,1.90123,1.9043,1.9074,
    1.8923,1.89519,1.89813,1.9011,1.90412,1.90719,1.91029,
    1.89524,1.89813,1.90106,1.90404,1.90706,1.91012,1.91323,
    1.89821,1.9011,1.90404,1.90701,1.91004,1.9131,1.91621,
    1.90123,1.90412,1.90706,1.91004,1.91306,1.91612,1.91923,
    1.9043,1.90719,1.91012,1.9131,1.91612,1.91918,1.92229,
    1.9074,1.91029,1.91323,1.91621,1.91923,1.92229,1.9254,
    0.699378,0.699378,0.699378,0.699378,0.699378,0.699378,0.699378,
    0.7,0.7,0.7,0.7,0.7,0.7,0.7,
    0.700624,0.700624,0.700624,0.700624,0.700624,0.700624,0.700624,
    0.701248,0.701248,0.701248,0.701248,0.701248,0.701248,0.701248,
    0.701873,0.701873,0.701873,0.701873,0.701873,0.701873,0.701873,
    0.702497,0.702497,0.702497,0.702497,0.702497,0.702497,0.702497,
    0.703119,0.703119,0.703119,0.703119,0.703119,0.703119,0.703119,
    0.699378,0.7,0.700624,0.701248,0.701873,0.702497,0.703119,
    0.699378,0.7,0.700624,0.701248,0.701873,0.702497,0.703119,
    0.699378,0.7,0.700624,0.701248,0.701873,0.702497,0.703119,
    0.699378,0.7,0.700624,0.701248,0.701873,0.702497,0.703119,
    0.699378,0.7,0.700624,0.701248,0.701873,0.702497,0.703119,
    0.699378,0.7,0.700624,0.701248,0.701873,0.702497,0.703119,
    0.699378,0.7,0.700624,0.701248,0.701873,0.702497,0.703119,
    0.600872,0.600436,0.599999,0.599561,0.599124,0.598687,0.598251,
    0.600436,0.6,0.599563,0.599125,0.598688,0.598251,0.597815,
    0.599999,0.599563,0.599126,0.598688,0.598251,0.597814,0.597378,
    0.599561,0.599125,0.598688,0.598251,0.597813,0.597376,0.59694,
    0.599124,0.598688,0.598251,0.597813,0.597376,0.596939,0.596503,
    0.598687,0.598251,0.597814,0.597376,0.596939,0.596502,0.596065,
    0.598251,0.597815,0.597378,0.59694,0.596503,0.596065,0.595629,
    1.25715,1.25715,2.22258
]

TT1 = vcat(TT0[1:NS*NS*3], TT0[end-2:end])

150-element Vector{Float64}:
 0.671248
 0.671248
 0.671248
 0.671248
 0.671248
 0.671248
 0.671248
 0.681248
 0.681248
 0.681248
 0.681248
 0.681248
 0.681248
 ⋮
 1.91918
 1.92229
 1.9074
 1.91029
 1.91323
 1.91621
 1.91923
 1.92229
 1.9254
 1.25715
 1.25715
 2.22258

In [272]:
φSYS(TT1, [0.1], [1,1,1,1,1])

150-element Vector{Float64}:
  4.670349880496808e-10
  6.902376024624273e-6
  1.3664605734057329e-5
  2.0294704396450164e-5
  2.6800946876202118e-5
  3.3191107094238704e-5
  3.947129976504999e-5
 -6.889611558000143e-6
  4.563570284866924e-10
  6.743478907329378e-6
  1.3347407574487724e-5
  1.982040874849658e-5
  2.6170382784145244e-5
  ⋮
  1.4564871339268863e-8
 -1.0320901024530454e-7
 -4.205654817908031e-6
 -2.876107011546181e-6
 -1.827070287962931e-6
 -1.021104337206927e-6
 -4.4933893871750095e-7
 -1.0320901024492507e-7
 -6.653387968065354e-10
 -4.6098090111801326e-7
 -4.6098090100699096e-7
  3.5477013272666724e-6



---

